> CNN512 + no dropout + hyperparameters tuning + luminocity(image enhancement + noise removal) + color normalization + data augmentation using IDRID original train test split (following the paper's instruction)

In [1]:
import pandas as pd
import numpy as np
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

luminocity

Image Enhancement: Two methods were used to enhance the images, the enhance luminosity method [54] and Contrast Limited Adaptive Histogram Equalization (CLAHE). CLAHE [55] is successful in enhancing the contrast of the fundus images [56] and improving the low contrast of medical images [57]. CLAHE is applied to the L channel of the retina images that have a higher contrast [44], with tile size 8 × 8 and Clip Limit 5.0.

Image Noise Removal: The CLAHE method can cause some noise in the images [54] and, to remove this noise, we applied the Gaussian filter

In [2]:
# Load the dataset
data = pd.read_csv("test_split_image_key.csv")

# Split the dataset into training and testing sets
train_data = data[data["split_type"] == "training"]
test_data = data[data["split_type"] == "testing"]

# Define output directories for preprocessed images
output_dir_train = "preprocessed_images_luminosity_new/train/"
output_dir_test = "preprocessed_images_luminosity_new/test/"

# Create directories if they don't exist
os.makedirs(output_dir_train, exist_ok=True)
os.makedirs(output_dir_test, exist_ok=True)

In [3]:
# Preprocessing function
def preprocess_image(image_path, output_dir):
    # Load and resize the image to 512x512
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (512, 512))

    # Enhance luminosity and apply Contrast Limited Adaptive Histogram Equalization (CLAHE)
    lab = cv2.cvtColor(resized_image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
    l_eq = clahe.apply(l)
    lab_eq = cv2.merge((l_eq, a, b))
    enhanced_image = cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)

    # Image noise removal using Gaussian filter
    filtered_image = gaussian_filter(enhanced_image, sigma=1)

    # Save the preprocessed image
    image_name = os.path.basename(image_path)
    output_path = os.path.join(output_dir, image_name)
    cv2.imwrite(output_path, filtered_image)


# Apply preprocessing to training and testing images
for img_name in train_data["Image name"]:
    image_path = "a_training/" + img_name + ".jpg"
    preprocess_image(image_path, output_dir_train)

for img_name in test_data["Image name"]:
    image_path = "b_testing/" + img_name + ".jpg"
    preprocess_image(image_path, output_dir_test)

In [4]:
# Convert the images to numpy arrays
X_train = []
X_test = []

for img_name in train_data["Image name"]:
    image_path = os.path.join(output_dir_train, img_name + ".jpg")
    image = cv2.imread(image_path)
    X_train.append(image)

for img_name in test_data["Image name"]:
    image_path = os.path.join(output_dir_test, img_name + ".jpg")
    image = cv2.imread(image_path)
    X_test.append(image)

X_train = np.array(X_train)
X_test = np.array(X_test)

# Extract labels directly without one-hot encoding
y_train = train_data["Retinopathy grade"].values
y_test = test_data["Retinopathy grade"].values

normalize

Colour Normalisation: The retina images were captured from patients of different age, and various ethnicity [58], at different levels of lighting in the fundus image. These conditions have an effect on the value of pixel intensity of each image and create unnecessary variation in the image [58]. To overcome this, the retina images were normalised by normalising each channel of RGB images. For the normalization, we subtract the mean, and after that, divide the variance of the images [25], as shown in Equation (2).

Augmentation

Online data augmentation was adopted to enlarge the training dataset and to improve the generalisation and performance of the CNN. The images were augmented by performing rotation, flipping, shearing, and translation, as well as randomly darkening and brightening them, as shown in Figure 8. The augmentation parameters are presented in Table 5. 

Data augmentation parameters.

Transformation Type	Description

Rotation	Rotate the image randomly between (−35∘, 35∘).

Flipping	Horizontal and vertical flip for the images.

Shearing	Randomly Shear images with angle between −15∘ and 15∘.

Translation	Randomly with shift between −10% and 10% of pixels.

Brightness range	Randomly darken the image and brighten. The values less than 1.0 the image darken whereas values larger than 1.0 brighten the image. The used values (0.25, 1.25).

In [5]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import math


# augmentation function
def augment(image, label):
    # Rotation (-35 degrees to 35 degrees)
    degrees = tf.random.uniform([], -35, 35, dtype=tf.float32)
    image = tfa.image.rotate(image, math.radians(degrees))

    # Flipping (randomly apply horizontal and vertical flipping)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    # Shearing (TensorFlow Addons needed)
    shearing = tf.random.uniform(
        [], -0.15, 0.15, dtype=tf.float32
    )  # Shear angle in radians
    image = tfa.image.shear_x(image, shearing, 0)  # Apply shearing in X direction
    image = tfa.image.shear_y(image, shearing, 0)  # Apply shearing in Y direction

    # Translation (randomly shift image within ±10% of its height and width)
    width_shift = tf.random.uniform([], -0.1, 0.1) * tf.shape(image)[1]
    height_shift = tf.random.uniform([], -0.1, 0.1) * tf.shape(image)[0]
    image = tfa.image.translate(image, [width_shift, height_shift])

    # Brightness (adjust brightness by a factor chosen from [0.25, 1.25])
    image = tf.image.random_brightness(
        image, max_delta=0.5
    )  # Note: delta adjusted to fit the [0.25, 1.25] range more closely

    return image, label


mean = np.mean(X_train, axis=(0, 1, 2))
std = np.std(X_train, axis=(0, 1, 2))


def normalize_image_with_label(image, label, mean, std):
    image = tf.cast(image, tf.float32)
    normalized_image = (image - mean) / (std + 1e-7)
    return normalized_image, label

ModuleNotFoundError: No module named 'tensorflow_addons'

In [ ]:
# Apply augmentation and normalization in data pipeline

# Preparing the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(
    lambda image, label: (augment(image), label), num_parallel_calls=tf.data.AUTOTUNE
)
# Applying the normalization function to a dataset
train_dataset = train_dataset.map(
    lambda image, label: normalize_image_with_label(image, label, mean, std),
    num_parallel_calls=tf.data.AUTOTUNE,
)

train_dataset = train_dataset.shuffle(1024).batch(32).prefetch(tf.data.AUTOTUNE)

# Preparing the testing dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.map(
    lambda image, label: normalize_image_with_label(image, label, mean, std),
    num_parallel_calls=tf.data.AUTOTUNE,
)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
input_shape = (512, 512, 3)
num_classes = 5
epochs = 50
batch_size = 32

> CNN 512 no dropout

## *CNN512* 

Layer	Operator	Layer Details

Input Layer	Zero Padding layer	Padding (2,2)

Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3

Layer 2	Batch Normalization layer	-

Layer 3	Relu layer	-

Layer 4	Max Pooling layer	Pooling size (2,2)

Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3

Layer 6	Batch Normalization layer	-

Layer 7	Relu layer	-

Layer 8	Max Pooling layer	Pooling size (2,2)

Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3

Layer 10	Batch Normalization layer	-

Layer 11	Relu layer	-

Layer 12	Max Pooling layer	Pooling size (2,2)

Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3

Layer 14	Batch Normalization layer	-

Layer 15	Relu layer	-

Layer 16	Max Pooling layer	Pooling size (2,2)

Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3

Layer 18	Batch Normalization layer	-

Layer 19	Relu layer	-

Layer 20	Max Pooling layer	Pooling size (2,2)

Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3

Layer 22	Batch Normalization layer	-

Layer 23	Relu layer	-

Layer 24	Max Pooling layer	Pooling size (2,2)

Layer 25	Flatten layer	-

Layer 26	FC layer	Neurons number = 1000

Layer 27	Batch Normalization layer	-

Layer 28	Relu layer	-

Layer 29	FC layer	Neurons number = 500

Layer 30	Batch Normalization layer	-

Layer 31	Relu layer	-

Layer 32	FC layer	With SoftMax activation


In [ ]:
cnn512_nodropout = models.Sequential(
    [
        keras.Input(shape=input_shape),
        # Input Layer	Zero Padding layer	Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2)),
        # Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3)),
        # Layer 2	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 3	Relu layer	-
        layers.Activation("relu"),
        # Layer 4	Max Pooling layer	Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3)),
        # Layer 6	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 7	Relu layer	-
        layers.Activation("relu"),
        # Layer 8	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 10	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 11	Relu layer	-
        layers.Activation("relu"),
        # Layer 12	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 14	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 15	Relu layer	-
        layers.Activation("relu"),
        # Layer 16	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3
        layers.Conv2D(128, kernel_size=(3, 3)),
        # Layer 18	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 19	Relu layer	-
        layers.Activation("relu"),
        # Layer 20	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3
        layers.Conv2D(200, kernel_size=(3, 3)),
        # Layer 22	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 23	Relu layer	-
        layers.Activation("relu"),
        # Layer 24	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 25	Flatten layer	-
        layers.Flatten(),
        # Layer 26	FC layer	Neurons number = 1000
        layers.Dense(1000),
        # Layer 27	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 28	Relu layer	-
        layers.Activation("relu"),
        # layers.Dropout(0.5),
        # Layer 29	FC layer	Neurons number = 500
        layers.Dense(500),
        # Layer 30	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 31	Relu layer	-
        layers.Activation("relu"),
        # layers.Dropout(0.5),
        # Layer 32	FC layer with Softmax activatio
        layers.Dense(num_classes, activation="softmax"),
    ]
)

## *The hyperparameter configuration of CNNs.*

Configuration	Values

Optimizer	SGD

Momentum	0.9

Max Learning rate	1×10−1 in custom CNNs

1×10−2 in EfficientNetB0 (exclude since not using EfficientNetB0)

Base Learning rate	1×10−4

Mode	triangular

Class weight	auto

Dropout	0.5 (exclude dropout)

Augmentation	20 times 

In [ ]:
from tensorflow.keras.optimizers import SGD

# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn512_nodropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

In [ ]:
import math

history_cnn512_nodropout = cnn512_nodropout.fit(
    train_dataset,
    epochs=epochs,
    steps_per_epoch=math.ceil(len(X_train) / 32),
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
    verbose=1,
)

In [ ]:
cnn512_nodropout.evaluate(test_dataset)